In [10]:
from flask import Flask, render_template, request ,send_file,send_from_directory
from geopy.geocoders import ArcGIS
import mysql.connector
from flask import Flask, render_template, request
from geopy.geocoders import ArcGIS
import geopandas as gpd
from werkzeug.utils import secure_filename
import os
import geojson
import asyncio

In [11]:
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'Uploadfile'
app.config['UPLOAD_FOLDER1'] = 'Uploadimage'

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="01156154345zA",
    database="company"
)

In [12]:
@app.route('/')
def home():
    cursor = mydb.cursor()

    # print data from data base
    cursor.execute("SELECT id,companies FROM companies")
    data = cursor.fetchall()

    return render_template('form.html', data=data)
    mydb.close()

In [13]:
@app.route('/poly/<int:company_id>')
def show_point_map1(company_id):

    
    cursor2 = mydb.cursor()
    cursor2.execute("SELECT id,companies FROM companies")
    data2 = cursor2.fetchall()

    
    cursor = mydb.cursor()
    cursor.execute("SELECT poly FROM vector WHERE id = %s", (company_id,))
    geojson_data = cursor.fetchone()[0]

    if geojson_data:
        data = geojson.loads(geojson_data)
    else:
        data = None

    return render_template("form.html", geojson_data=data, company_id=company_id,data=data2)

@app.route('/point/<int:company_id>')
def show_point_map2(company_id):
    
    cursor2 = mydb.cursor()
    cursor2.execute("SELECT id,companies FROM companies")
    data2 = cursor2.fetchall()


    cursor = mydb.cursor()
    cursor.execute("SELECT point FROM vector WHERE id = %s", (company_id,))
    geojson_data = cursor.fetchone()[0]

    if geojson_data:
        data = geojson.loads(geojson_data)
    else:
        data = None

    return render_template("form.html", geojson_data=data, company_id=company_id,data=data2)

@app.route('/line/<int:company_id>')
def show_point_map3(company_id):
    cursor2 = mydb.cursor()
    cursor2.execute("SELECT id,companies FROM companies")
    data2 = cursor2.fetchall()

    
    cursor = mydb.cursor()
    cursor.execute("SELECT line FROM vector WHERE id = %s", (company_id,))
    geojson_data = cursor.fetchone()[0]

    if geojson_data:
        data = geojson.loads(geojson_data)
    else:
        data = None

    return render_template("form.html", geojson_data=data, company_id=company_id,data=data2)

In [14]:
@app.route("/uploadfile/<filename>")
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

@app.route("/upload", methods=["POST", "GET"])
def Vector():
    error="No File Uploaded"
    cursor2 = mydb.cursor()
    cursor2.execute("SELECT id,companies FROM companies")
    data = cursor2.fetchall()
    if request.method == "POST":
        file = request.files['Vector']
        if file.filename == '':
            return render_template("form.html",error1=error, data=data)

        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        
        # Pass the filename to the template
        return render_template("form.html",error=error, filename=filename,data=data)
    
    return render_template("form.html",data=data)

In [15]:
@app.route('/search', methods=['POST'])
def search():
    cursor2 = mydb.cursor()
    cursor2.execute("SELECT id,companies FROM companies")
    data = cursor2.fetchall()
    if request.method == 'POST':
        latitude = request.form['latitude']
        longitude = request.form['longitude']
        if not latitude or not longitude:
            return render_template('form.html', error_message='Please enter latitude and longitude.',data=data)
        return render_template('form.html', latitude=latitude, longitude=longitude,data=data)

In [16]:
@app.route("/company",methods=["Post"])
def company():
    
    cursor2 = mydb.cursor()
    cursor2.execute("SELECT id,companies FROM companies")
    data = cursor2.fetchall()
    error_company="Company not found"
    
    cursor1=mydb.cursor()
    cursor1.execute("select companies from companies")
    company=cursor1.fetchall()
    
    cursor=mydb.cursor()
    search_company = request.form['company_name']
    sql_query = "SELECT companies FROM companies WHERE companies = %s"
    cursor.execute(sql_query, (search_company,))
    result=cursor.fetchone()

    
    if result:
        address = result[0]  
        geolocator = ArcGIS()
        location = geolocator.geocode(address)
        if location:
            return render_template("form.html",  latitude=location.latitude, longitude=location.longitude,data=data)
        else:
                return render_template("form.html", error_company=error_company,data=data)

    return render_template("form.html", error_company=error_company,company=company,data=data)

In [17]:
@app.route("/Login")
def login():
    return render_template("Login.html")

In [18]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Dec/2023 10:22:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 10:22:13] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Dec/2023 10:22:13] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Dec/2023 10:22:13] "GET /static/index.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Dec/2023 10:22:13] "GET /static/images/background.webp HTTP/1.1" 304 -
127.0.0.1 - - [17/Dec/2023 10:41:17] "POST /company HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 10:41:17] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Dec/2023 10:41:17] "GET /static/index.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Dec/2023 10:41:17] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Dec/2023 10:41:17] "GET /static/images/background.webp HTTP/1.1" 304 -
127.0.0.1 - - [17/Dec/2023 10:41:34] "GET /poly/1 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 10:41:34] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Dec/2023 10